In [1]:
import nltk
import numpy as np
import pandas as pd
from gensim.models import LdaModel
import gensim
from gensim.corpora import Dictionary, MmCorpus
import gensim.corpora as corpora
import glob

import re
import numpy as np
import pandas as  pd
from pprint import pprint# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel# spaCy for preprocessing
import spacy# Plotting tools
import pyLDAvis
import pyLDAvis.gensim_models
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
from wordcloud import WordCloud

import nltk
nltk.download('stopwords')

/opt/homebrew/lib/python3.10/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  from imp import reload
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/lukasburtscher/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
df = pd.read_csv('../../data/feature/clusters/0_cluster.csv')
df.head()

,ID_GodotObject,cluster_value,merged_text
0,2000116569218,0,﻿20_Coronavirus ab sofort einkaufen maskieren ...
1,2000118306572,0,﻿20 genießen Verantwortung Urlaub beachten sol...
2,2000119010979,0,﻿20tagesüberblick corona corona -cluster st Wo...
3,2000119396183,0,﻿20_Coronavirus kroatisch Botschafter hoffen p...
4,2000120059197,0,﻿20tagesüberblick_Coronavirus versch ärfter co...


In [3]:
# Merge all text per cluster
# Find all CSV files in the current directory
filenames = glob.glob('../../data/feature/clusters/*.csv')

# Create a list to store the dataframes
df_clusters = pd.DataFrame()
# Define a function that concatenates the text from a row
def concat_text(row):
    return ' '.join(row)

# Loop over the CSV files
for filename in filenames:
    # Read the file into a dataframe
    
    df = pd.read_csv(filename)
    l = ''.join((df['merged_text'].to_numpy()))
    c = df['cluster_value'][0]
    df_clusters = df_clusters.append({'cluster' : c, 'text' : l}, ignore_index = True)
    
df_clusters.sort_values(by=['cluster'], inplace=True)
df_clusters.reset_index(drop=True, inplace=True)
df_clusters.head()

/var/folders/1_/83qhk9ps6fd0yym23nn9nfz00000gn/T/ipykernel_7009/2095982231.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_clusters = df_clusters.append({'cluster' : c, 'text' : l}, ignore_index = True)
/var/folders/1_/83qhk9ps6fd0yym23nn9nfz00000gn/T/ipykernel_7009/2095982231.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_clusters = df_clusters.append({'cluster' : c, 'text' : l}, ignore_index = True)
/var/folders/1_/83qhk9ps6fd0yym23nn9nfz00000gn/T/ipykernel_7009/2095982231.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_clusters = df_clusters.append({'cluster' : c, 'text' : l}, ignore_index = True)
/var/folders/1_/83qhk9ps6fd0yym23nn9nfz00000gn/T/ipykernel_7009/2095982231.py:18: FutureWarning: The

,cluster,text
0,0,﻿20_Coronavirus ab sofort einkaufen maskieren ...
1,1,﻿20corona-regeln15-jährige klagen Republik weg...
2,2,﻿20_Coronavirus Sonderbeauftragter Clemens Aue...
3,3,﻿20_Coronavirus schrittweise Einführung Masken...
4,4,﻿20_Coronavirus Maskenpflicht Supermarkt späte...


It is generally not necessary to remove duplicate words from a text before calculating the term frequency-inverse document frequency (TF-IDF) scores for the words. The TF-IDF score of a word takes into account both the frequency of the word in the document (term frequency, or TF) and the inverse of the frequency of the word in the collection (inverse document frequency, or IDF).

For example, consider the following two documents:

Document 1: "the cat sat on the mat"
Document 2: "the cat sat on the mat"

The term frequency of "the" in both documents is 2, because it appears twice in each document. The inverse document frequency of "the" in this collection is 1, because it appears in both documents. Therefore, the TF-IDF score of "the" in both documents is 2 * 1 = 2.

On the other hand, the term frequency of "cat" in both documents is 1, because it appears once in each document. The inverse document frequency of "cat" in this collection is 1, because it appears in both documents. Therefore, the TF-IDF score of "cat" in both documents is 1 * 1 = 1.

As you can see, the TF-IDF scores of the words in the documents are the same, regardless of whether the words are duplicated or not. Therefore, it is not necessary to remove duplicate words from a text before calculating the TF-IDF scores.

In [4]:
#TF IDS Score for labeling the clusters
# settings that you use for count vectorizer will go here 
tfidf_vectorizer=TfidfVectorizer(use_idf=True) 
# just send in all your docs here 
tfidf_vectorizer_vectors=tfidf_vectorizer.fit_transform(df_clusters['text'])

# Convert the sparse matrix to a dense array
dense_array = tfidf_vectorizer_vectors.toarray()

# Get the feature names from the TfidfVectorizer object
feature_names = tfidf_vectorizer.get_feature_names()

# Create a dataframe from the dense array
df = pd.DataFrame(dense_array, columns=feature_names).T.reset_index()


df.head()



/opt/homebrew/lib/python3.10/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,index,0,1,2,3,4,5,6
0,0000,0.0,0.0,0.000000,0.0000,0.000000,0.0,0.006696
1,006121,0.0,0.0,0.010533,0.0000,0.000000,0.0,0.000000
2,006721,0.0,0.0,0.010533,0.0000,0.000000,0.0,0.000000
3,024,0.0,0.0,0.000000,0.0000,0.009219,0.0,0.000000
4,033,0.0,0.0,0.000000,0.0123,0.000000,0.0,0.000000


In [5]:
dfs = []

for col in df:
    if col == 'index':
        continue;
    else:
        dfcol = df[['index',col]].copy().sort_values(by=col, ascending=False).reset_index(drop=True)
        dfcol.columns = ['words_'+str(col), 'tfidf_'+str(col)]
        wordcloud = dfcol.set_index('words_'+str(col))
        dfs.append(dfcol)


#print(len(dfs))

In [6]:
dfs[3].head()

,words_3,tfidf_3
0,österreich,0.222025
1,coronavirus,0.222025
2,person,0.204497
3,corona,0.170098
4,werden,0.144325


In [7]:
result = pd.concat(dfs,axis=1)
result.head()

,words_0,tfidf_0,words_1,tfidf_1,words_2,tfidf_2,words_3,tfidf_3,words_4,tfidf_4,words_5,tfidf_5,words_6,tfidf_6
0,gelten,0.211284,haimbuchner,0.501710,sein,0.215146,österreich,0.222025,ab,0.214585,corona,0.262439,corona,0.258156
1,sein,0.201532,storchenfeier,0.250855,ffp2,0.153463,coronavirus,0.222025,corona,0.212487,maskenpflicht,0.192813,wien,0.182393
2,ab,0.195031,geldstrafe,0.250855,mehr,0.140095,person,0.204497,werden,0.212487,werden,0.160677,gelten,0.178121
3,corona,0.180659,klage,0.209046,corona,0.132420,corona,0.170098,wien,0.200897,vfgh,0.154162,ab,0.178121
4,geben,0.151983,corona,0.192726,maskenpflicht,0.128006,werden,0.144325,polizei,0.189044,wien,0.123186,werden,0.173975


In [8]:
result.to_csv('../../data/feature/tfids_cluster.csv', encoding='utf-8', index=False)

In [9]:
## WORDCLOUDS
#-- 5 Topics
# 0 = Regel, Schule, Impfung, Schutz
# 1 = Haimbuchner, Griechenland, Gesetzwidrig, FPÖ, Klage, angefochten
# 2 = Polizei, Demo, Neuinfektionen
# 3 = Maßnahmen verschärfen
# 4 = Biden, USA, ablenkend

#-- 7 topics

# 0 = Maskenpflicht, Regel, Österreich weit, FFP2
# 1 = Haimbuchner, Klage, Geldstrafe
# 2 = Schule, Maskenpflicht, Kinder
# 3 = Coronavirus aktuelle Zahlen, Hinweis, 0800 555
# 4 = Polizei, Demonstrationen
# 5 = Verfassungsgerichtshof, Griechenland, Delta Variante
# 6 = Corona Ampel Gelb, Veranstaltungen, Mückstein, regierung